In [1]:
# notebook gives the right result, but this gives errors
# todo: it gets the wrong movies
# todo: index to excell must be removed
# todo: excel must be ordered
# todo: is adult must be removed

import os
import sys
import time
import numpy as np
import pandas as pd
from datetime import date
from datetime import timedelta

START_TIME = time.time()

BASE_URL = "https://datasets.imdbws.com/"
PARQ_PATH = "data/imdb/parquet/"
DOWNLOAD_PATH = "data/imdb/download/"
OUTPUT_PATH = "data/generated/"


FILES_IMDB = {
    "cast_crew" : "title.crew.tsv",
    "tit_bas" : "title.basics.tsv",
    "tit_rate" : "title.ratings.tsv",
    "name_bas" : "name.basics.tsv",
    "tit_prin" : "title.principals.tsv",
}

FILES_IMDB_PARQ = {
    "tit_bas" : "title_basics.parquet",
    "genres" : "genres.parquet",
    "tit_rate" : "title_ratings.parquet",
    'directors' : 'directors.parquet',
    'writers' : 'writers.parquet',
    'prim_prof' : 'primary_profession.parquet',
    'known_for' : 'known_for_titles.parquet',
    'name_bas' : 'name_basics.parquet',
    'const' : 'ids.parquet',
    'ordering' : 'ordering.parquet',
    'character' : 'character.parquet',
    'job' : 'job.parquet',
}

FILES_HAND = {
    "add_seen": "add_movies_seen.txt",
    "add_unseen": "add_movies_unseen.txt",
    "add_secop": "add_movies_second_opinion.txt",
    "raw_status": "raw_status.xlsx"
}

FILES_GENERATED = {
    "films_raw": "films_raw.pkl",
    "films_reading": "films_reading.xlsx"
}

In [2]:
class AppendedMovieList():
    BASE_URL = "https://datasets.imdbws.com/"
    PARQ_PATH = "data/imdb/parquet/"
    DOWNLOAD_PATH = "data/imdb/download/"

    def __init__(self,
                 url_movie_list,
                 url_imdb_ids,
                 url_title_basics,
                 url_title_rate,
                 url_genre,
                 url_directors,
                 url_writers,
                 url_personnel,
                 url_name_basics
                 ):
        
        self.__movie_list = pd.DataFrame()
        self.url_movie_list = url_movie_list
        self.url_imdb_ids = url_imdb_ids
        self.url_title_basics = url_title_basics
        self.url_title_rate = url_title_rate
        self.url_genre = url_genre
        self.url_directors = url_directors
        self.url_writers = url_writers
        self.url_personnel = url_personnel
        self.url_name_basics = url_name_basics



        # self._loadList()
        # self._addMovieIds()
        # self._addTitleBasics()
        # self._addRating()
        # self._addGenre()
        # self._addPersonnel()


    def loadList(self):
        # load the raw movie list
        self.__movie_list = pd.read_excel(self.url_movie_list)


    def addMovieIds(self):
        # load movieID's
        const = pd.read_parquet(self.url_imdb_ids)
        
        # add movieID's
        self.__movie_list = pd.merge(
            self.__movie_list, const[const["type"]=="tconst"]["intid"],
            how='left', left_on="tconst", right_index=True)
        self.__movie_list.set_index('intid', drop=True)
    

    def addTitleBasics(self):
        # load title basics
        tit_bas = pd.read_parquet(self.url_title_basics)

        # add title basics
        self.__movie_list = pd.merge(self.__movie_list, tit_bas,
                                     how='left', left_index=True, right_index=True)
    

    def addRating(self):
        # load ratings
        tit_rate = pd.read_parquet(self.url_title_rate)

        # add title rate
        self.__movie_list = pd.merge(self.__movie_list, tit_rate, how='left',
                                  left_index=True, right_index=True)


    def addGenre(self):
        # load genres
        genre = pd.read_parquet(self.url_genre)

        # get needed ones
        genre = genre[genre.index.isin(self.__movie_list.index)]
        genre['genre'] = genre['genre'].astype('category')

        # convert to multi one-hot
        genre = pd.crosstab(genre.index, genre['genre'])
        
        # add genre
        self.__movie_list = pd.merge(self.__movie_list, genre, how='left', left_index=True, right_index=True)


    def loadPersonnel(self):
        # load neaded directors
        directors = pd.read_parquet(self.url_directors)
        directors = directors[directors.index.isin(self.__movie_list.index)]
        directors['category'] = 'director'
        directors = directors.rename(columns={'directors':'nconst'})

        # load needed writers
        writers = pd.read_parquet(self.url_writers)
        writers = writers[writers.index.isin(self.__movie_list.index)]
        writers['category'] = 'writer'
        writers = writers.rename(columns={'writers':'nconst'})

        # load needed ordering
        personnel = pd.read_parquet(self.url_personnel)
        personnel = personnel[personnel.index.isin(self.__movie_list.index)].drop(columns='ordering')

        # combine personell id's
        all_personnel = pd.concat([writers, directors, personnel])
        all_personnel = all_personnel.drop_duplicates()
        all_personnel.loc[:,'category'] = all_personnel.loc[:,'category'].astype('category')

        return all_personnel
    
    
    def addPersonnel(self):
        # prepare personell ids
        all_personnel = self._loadPersonnel()

        # get needed personell info
        name_bas = pd.read_parquet(self.url_name_basics)
        names = name_bas[name_bas.index.isin(all_personnel['nconst'])]

        # convert personell info
        names = names["primaryName"] + " (" + names["birthYear"].astype(str).replace("<NA>","") + "-" + names["deathYear"].astype(str).replace("<NA>","") + ")"
        names.name = "info"

        # add personell info to movie indices
        personell = pd.merge(all_personnel, names, how='left', left_on='nconst', right_index=True).drop(columns='nconst')

        # convert personell categories to columns
        personell = pd.pivot_table(personell, values='info', index=['tconst'], columns=['category'], aggfunc=list)

        # add personell to watched movies (can probably be replaced by a normal add)
        self.__movie_list = pd.merge(self.__movie_list, personell, how='left', left_index=True, right_index=True)


    def getMovieList(self):
        return self.__movie_list

In [3]:

# movie_list_raw = loadList()
# movie_list = addMovieIds(movie_list_raw)
# movie_list = addTitleBasics(movie_list)
# movie_list = addRating(movie_list)

id_stat =       os.path.join("data", "handcrafted", FILES_HAND["raw_status"])
const =         os.path.join(PARQ_PATH, FILES_IMDB_PARQ["const"])
tit_bas =       os.path.join(PARQ_PATH, FILES_IMDB_PARQ["tit_bas"])
tit_rat =       os.path.join(PARQ_PATH, FILES_IMDB_PARQ["tit_rate"])
genre =         os.path.join(PARQ_PATH, FILES_IMDB_PARQ["genres"])
directors =     os.path.join(PARQ_PATH, FILES_IMDB_PARQ["directors"])
writers =       os.path.join(PARQ_PATH, FILES_IMDB_PARQ["writers"])
personnel =     os.path.join(PARQ_PATH, FILES_IMDB_PARQ["ordering"])
name_basics =   os.path.join(PARQ_PATH, FILES_IMDB_PARQ["name_bas"])
# output_excel =   os.path.join(OUTPUT_PATH, FILES_GENERATED["films_reading"])
output_excel =   os.path.join(OUTPUT_PATH, "test.xlsx")

test = AppendedMovieList(
    url_movie_list = id_stat,
    url_imdb_ids = const,
    url_title_basics = tit_bas,
    url_title_rate = tit_rat,
    url_genre = genre,
    url_directors = directors,
    url_writers = writers,
    url_personnel =  personnel,
    url_name_basics = name_basics
    )

movies = test.getMovieList()

# movies.to_excel(output_excel)

# END_TIME = time.time()
# time_format = time.strftime("%H:%M:%S", time.gmtime(END_TIME-START_TIME))
# print("Execution time: ",time_format)

In [4]:
test.loadList()

In [5]:
test.getMovieList()

,tconst,watched,watched_date,netflix,prime,enjoyment,priority
0,tt0015324,0,NaT,NaN,NaN,NaN,NaN
1,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN
2,tt0022100,0,NaT,0.0,0.0,NaN,NaN
3,tt0025316,0,NaT,NaN,NaN,NaN,NaN
4,tt0031381,0,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
621,tt9691136,0,NaT,0.0,0.0,NaN,NaN
622,tt9731598,0,NaT,0.0,0.0,NaN,NaN
623,tt9764362,1,NaT,0.0,0.0,4.0,NaN
624,tt9783600,1,NaT,NaN,NaN,4.0,NaN


In [6]:
test.addMovieIds()

In [7]:
test.getMovieList()

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,intid
0,tt0015324,0,NaT,NaN,NaN,NaN,NaN,15324
1,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN,17136
2,tt0022100,0,NaT,0.0,0.0,NaN,NaN,22100
3,tt0025316,0,NaT,NaN,NaN,NaN,NaN,25316
4,tt0031381,0,NaT,NaN,NaN,NaN,NaN,31381
...,...,...,...,...,...,...,...,...
621,tt9691136,0,NaT,0.0,0.0,NaN,NaN,9691136
622,tt9731598,0,NaT,0.0,0.0,NaN,NaN,9731598
623,tt9764362,1,NaT,0.0,0.0,4.0,NaN,9764362
624,tt9783600,1,NaT,NaN,NaN,4.0,NaN,9783600


In [8]:
test.addTitleBasics()

In [9]:
test.getMovieList()

,tconst,watched,watched_date,netflix,prime,enjoyment,priority,intid,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
0,tt0015324,0,NaT,NaN,NaN,NaN,NaN,15324,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN,17136,short,Carmencita,Carmencita,False,1894,<NA>,1
2,tt0022100,0,NaT,0.0,0.0,NaN,NaN,22100,short,Le clown et ses chiens,Le clown et ses chiens,False,1892,<NA>,5
3,tt0025316,0,NaT,NaN,NaN,NaN,NaN,25316,short,Pauvre Pierrot,Pauvre Pierrot,False,1892,<NA>,4
4,tt0031381,0,NaT,NaN,NaN,NaN,NaN,31381,short,Un bon bock,Un bon bock,False,1892,<NA>,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,tt9691136,0,NaT,0.0,0.0,NaN,NaN,9691136,short,That Fatal Sneeze,That Fatal Sneeze,False,1907,<NA>,6
622,tt9731598,0,NaT,0.0,0.0,NaN,NaN,9731598,short,Tierra baja,Tierra baja,False,1907,<NA>,20
623,tt9764362,1,NaT,0.0,0.0,4.0,NaN,9764362,short,The Magic Bag,Tryllesækken,False,1907,<NA>,4
624,tt9783600,1,NaT,NaN,NaN,4.0,NaN,9783600,short,The Ugly Duckling,The Ugly Duckling,False,1907,<NA>,<NA>


In [10]:
    
movie_list = pd.read_excel(id_stat)

# load movieID's
const_f = pd.read_parquet(const)

In [11]:
const_f

,intid,type
stringid,,
tt0000001,1,tconst
tt0000002,2,tconst
tt0000003,3,tconst
tt0000004,4,tconst
tt0000005,5,tconst
...,...,...
nm9993714,9993714,nconst
nm9993716,9993716,nconst
nm9993717,9993717,nconst


In [12]:
# add movieID's
movie_list = pd.merge(
    movie_list, const_f[const_f["type"]=="tconst"]["intid"],
    how='left', left_on="tconst", right_index=True)
movie_list = movie_list.set_index('intid', drop=True)

In [13]:
movie_list

,tconst,watched,watched_date,netflix,prime,enjoyment,priority
intid,,,,,,,
15324,tt0015324,0,NaT,NaN,NaN,NaN,NaN
17136,tt0017136,1,2023-03-04,0.0,0.0,3.5,NaN
22100,tt0022100,0,NaT,0.0,0.0,NaN,NaN
25316,tt0025316,0,NaT,NaN,NaN,NaN,NaN
31381,tt0031381,0,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
9691136,tt9691136,0,NaT,0.0,0.0,NaN,NaN
9731598,tt9731598,0,NaT,0.0,0.0,NaN,NaN
9764362,tt9764362,1,NaT,0.0,0.0,4.0,NaN


In [ ]:
# load title basics
tit_bas = pd.read_parquet(tit_bas)

In [ ]:
# add title basics
movie_list = pd.merge(movie_list, tit_bas,
                                how='left', left_index=True, right_index=True)